In [57]:
import pandas as pd
import pathlib as path

In [111]:
rel_path = path.Path().absolute()
od = pd.read_csv(str(rel_path) + r"\raw_data\Oscars-demographics-DFE_utf_8.csv")
ids = pd.read_csv(str(rel_path) + r"\raw_data\dfe_movieID.tsv", sep = '\t')

del ids['Unnamed: 0']
del od['_trusted_judgments']
del od['_last_judgment_at']

In [112]:
ids_drop = ids.drop_duplicates()
df = od.merge(ids_drop, on = ['movie', 'year_of_award'])
del df['_unit_id']
del df['_golden']
del df['_unit_state']
del df['date_of_birth:confidence']
del df['race_ethnicity_gold']
del df['religion_gold']
del df['sexual_orientation_gold']
del df['year_of_award_gold']
del df['race_ethnicity:confidence']
del df['sexual_orientation:confidence']
del df['date_of_birth_gold']
del df['birthplace_gold']
del df['birthplace:confidence']
del df['year_of_award:confidence']
df['movie_IMDB_Link'] = df['movieID']

In [113]:
# df['imdbID'] = df['biourl'].str.split('/', expand = True)[4]
df['movieID'] = df['movieID'].str.split('/', expand = True)[4]

In [115]:
df.to_csv(str(rel_path) + r'\..\src\oscar_demos_def.csv')